In [ ]:
# initialize Spark
from pyspark import SparkContext, SparkConf
if not 'sc' in globals():
    conf = SparkConf().setMaster('local[*]')
    sc = SparkContext(conf=conf)

In [ ]:
reads = sc.textFile("file:////home/jovyan/work/data/blast/input/sample.fa")

In [ ]:
print reads.take(2)

In [ ]:
indexedReads = reads.zipWithIndex()
print indexedReads.take(2)

In [ ]:
keyedReads = indexedReads.map(lambda tup: (tup[0], tup[1] if tup[1] % 2 == 0 else tup[1] - 1)).map(lambda tup: (tup[1], tup[0]))

In [ ]:
print keyedReads.take(2)

In [ ]:
metaAndSeq = keyedReads.groupByKey().map(lambda tup: (tup[0], list(tup[1])))

In [ ]:
print metaAndSeq.take(1)
print metaAndSeq.count()

In [ ]:
filtered = metaAndSeq.filter(lambda tup: len(tup[1]) == 2)
print filtered.count()

In [ ]:
inputSeqs = metaAndSeq.map(lambda tup: tup[1][0] + "\n" + tup[1][1])
print inputSeqs.take(1)[0]

In [ ]:
script = "/home/jovyan/work/data/blast/program/ncbi-blast-2.4.0+/runblast"
blastedSeqs = inputSeqs.pipe(script)
print blastedSeqs.take(10)
blastedSeqs.cache()

In [ ]:
taxIn = sc.textFile("file://///home/jovyan/work/data/blast/tax/gi_taxid_prot.dmp")
tax = taxIn.map(lambda s: tuple(s.split())).cache()

In [ ]:
print tax.first()

In [ ]:
giIds = blastedSeqs.map(lambda s: (s.split()[1].split("|")[1], s.split()[0])).cache()
print giIds.first()

In [ ]:
joinedTaxId = giIds.join(tax).cache()
print joinedTaxId.take(2)

In [ ]:
namesIn  = sc.textFile("file://///home/jovyan/work/data/blast/tax/names.dmp")
names = namesIn.map(lambda s : tuple(s.split("\t", 1))).cache()

In [ ]:
joinedNamesId = joinedTaxId.join(names)
joinedNamesId.take(3)